In [ ]:
!pip install python-telegram-bot --upgrade

In [ ]:
import random 
import nltk
import json
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [ ]:
with open('/content/BOT_CONFIG.json') as f:
  BOT_CONFIG = json.load(f)

In [ ]:
def clean(text):
  clean_text = ''
  for ch in text.lower():
    if ch in 'абвгдеёжзийклмнопрстуфхцчшщъыьэюя':
      clean_text = clean_text + ch
    return clean_text

In [ ]:
x = []
y = []
for intent in BOT_CONFIG['intents'].keys():
  for example in BOT_CONFIG['intents'][intent]['examples']:
    x.append(example)
    y.append(intent)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x,y,random_state=42, test_size=0.3)

In [ ]:
vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(x_train)
X_test = vectorizer.transform(x_test)

In [ ]:
vectname = vectorizer.get_feature_names_out()
len(vectname)

749

In [ ]:
clf = RandomForestClassifier(n_estimators=300).fit(X_train,y_train)
clf.score(X_test,y_test)

0.07605633802816901

In [ ]:
def get_intent_by_model(text):
  return clf.predict(vectorizer.transform([text]))[0] 

In [ ]:
def bot(input_text):
  intent = get_intent_by_model(input_text)
  return random.choice(BOT_CONFIG['intents'][intent]['responses'])

In [ ]:
input_text = ''
while input_text != 'stop':
  input_text = input('>')
  if input_text != 'stop':
    response = bot(input_text)
    print(response)

>stop


In [ ]:
import logging

from telegram import Update, ForceReply
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters, CallbackContext

# Enable logging
logging.basicConfig(
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', level=logging.INFO
)

logger = logging.getLogger(__name__)


# Define a few command handlers. These usually take the two arguments update and
# context.
def start(update: Update, context: CallbackContext) -> None:
    """Send a message when the command /start is issued."""
    user = update.effective_user
    update.message.reply_markdown_v2(
        fr'Hi {user.mention_markdown_v2()}\!',
        reply_markup=ForceReply(selective=True),
    )


def help_command(update: Update, context: CallbackContext) -> None:
    """Send a message when the command /help is issued."""
    update.message.reply_text('Help!')


def echo(update: Update, context: CallbackContext) -> None:
    """User message.""" # Меняем эхо бота 
    input_text = update.message.text
    output_text = bot(input_text)
    update.message.reply_text(output_text)


def main() -> None:
    """Start the bot."""
    # Create the Updater and pass it your bot's token.
    updater = Updater("token bot")

    # Get the dispatcher to register handlers
    dispatcher = updater.dispatcher

    # on different commands - answer in Telegram
    dispatcher.add_handler(CommandHandler("start", start))
    dispatcher.add_handler(CommandHandler("help", help_command))

    # on non command i.e message - echo the message on Telegram
    dispatcher.add_handler(MessageHandler(Filters.text & ~Filters.command, echo))

    # Start the Bot
    updater.start_polling()

    # Run the bot until you press Ctrl-C or the process receives SIGINT,
    # SIGTERM or SIGABRT. This should be used most of the time, since
    # start_polling() is non-blocking and will stop the bot gracefully.
    updater.idle()


if __name__ == '__main__':
    main()

2022-01-12 18:41:20,813 - apscheduler.scheduler - INFO - Scheduler started
